In [4]:
! pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
from nltk.metrics.agreement import AnnotationTask
from nltk.metrics import masi_distance
import pandas as pd
import os
from sklearn.metrics import jaccard_score
import numpy as np

In [13]:
high_level_mapping = {
    'abbreviation without context':'lack of context',
    'unclear intention': 'lack of context',
    'write up': 'content generation',
    'generate persona': 'content generation',
    'exam reference': 'content generation',
    'generate question': 'content generation',
    'generate code template': 'content generation',
    'code': 'content generation',
    'generate test': 'content generation',
    'elaboration': 'content generation',
    'project assistance': 'content generation',
    'summarise topic': 'summarisation',
    'summarise literature': 'summarisation',
    'summarize topic': 'summarisation',
    'summarize literature': 'summarisation',
    'summarization': 'summarisation',
    'definition': 'understand concept',
    'understand': 'understand concept',
    'application': 'understand concept',
    'assistance in using application': 'it assistance',
    'technical documentation': 'it assistance',
    'set up': 'it assistance',
    'debugging': 'programming assistance',
    'code validation': 'programming assistance',
    'code suggestion': 'programming assistance',
    'writing refinement': 'language assistance',
    'sensitive writing': 'language assistance',
    'sutd-based information': 'search engine',
    'retrieve data': 'search engine',
    'find examples': 'search engine',
    'industry knowledge': 'search engine',
    'search engine query': 'search engine',
    'avoid plagiarism detection': 'academic dishonesty',
    'attempt to obtain answer for an assignment verbatim': 'academic dishonesty',
    'possible academic dishonesty': 'academic dishonesty',
    'sanity check': 'testing',
    'testing capabilities': 'testing',
    'gptlearn technical specification': 'clarify',
    'role-playing': 'prompt engineering',
    'establishing context': 'prompt engineering',
    'structured output format': 'prompt engineering',
    'idea validation': 'evaluation'
}

In [17]:
def modify_list(codes_list, code_dependency):
    codes_list = [code_dependency.get(item.lower(), item.lower()) for item in codes_list]
    codes_list = list(set(codes_list))
    # for key, value in code_dependency.items():
    #     if key in codes_list and value not in codes_list:
    #         codes_list.append(value)
    return codes_list

all_data = []
for file in os.listdir('ENCODED'):
    if file.endswith('.xlsx') and not file.startswith('~'):
        print(file)
        df = pd.read_excel(os.path.join('ENCODED', file))
        df = df.sort_values(by=['document', 'quotation'])
        df = df.drop(columns=['comment'])
        df['codes'] = df['codes'].fillna('')
        df['codes'] = df['codes'].astype(str)
        df = df.groupby('document').agg({
            'codes': ', '.join
        }).reset_index()
        df['codes'] = df['codes'].str.split(', ')
        df['codes'] = df['codes'].apply(lambda x: modify_list(x, high_level_mapping))
        df['codes'] = df['codes'].apply(sorted)
        all_data.append(df)

GPTLearnCoding-Keith.xlsx
GPTLearn Coding 1 - Daniel.xlsx
GPTLearn Coding 1 - Ivan.xlsx


In [22]:
merged_df = all_data[0].rename(columns={'codes': 'coder1'})

for idx, df in enumerate(all_data[1:], 2):  # Start from the second DataFrame and the second coder
    merged_df = merged_df.merge(df.rename(columns={'codes': f'coder{idx}'}), on='document', how='outer')

merged_df['coder1'] = merged_df['coder1'].fillna('[]')
merged_df = merged_df.set_index('document')
merged_df.head()

,coder1,coder2,coder3
document,,,
1.docx,[understand concept],"[idea generation, prompt engineering, understa...",[content generation]
10.docx,"[academic dishonesty, non-course related]","[academic dishonesty, non-course related]","[academic dishonesty, non-course related]"
100.docx,"[it assistance, understand concept]","[it assistance, prompt engineering, understand...",[it assistance]
1000.docx,"[non-course related, understand concept]","[non-course related, understand concept]",[testing]
1001.docx,"[content generation, prompt engineering]","[academic dishonesty, content generation, non-...","[academic dishonesty, content generation, non-..."


In [26]:
# All lvl1 code dict, to count the Expected Agreement if Random Judgement
lvl1_code_count = {}

for lvl1Label in high_level_mapping.keys():
    lvl1_code_count[lvl1Label] = 0

for idx, row in merged_df.iterrows():
    row['coder1']


coder1                                 [understand concept]
coder2    [idea generation, prompt engineering, understa...
coder3                                 [content generation]
Name: 1.docx, dtype: object
coder1    [academic dishonesty, non-course related]
coder2    [academic dishonesty, non-course related]
coder3    [academic dishonesty, non-course related]
Name: 10.docx, dtype: object
coder1                  [it assistance, understand concept]
coder2    [it assistance, prompt engineering, understand...
coder3                                      [it assistance]
Name: 100.docx, dtype: object
coder1    [non-course related, understand concept]
coder2    [non-course related, understand concept]
coder3                                   [testing]
Name: 1000.docx, dtype: object
coder1             [content generation, prompt engineering]
coder2    [academic dishonesty, content generation, non-...
coder3    [academic dishonesty, content generation, non-...
Name: 1001.docx, dtype: object
co